In [1]:
import numpy as np 
import networkx as nx
import matplotlib.pyplot as plt
from vbi.models.cpp.mpr import MPR_sde

In [2]:
seed = 2
np.random.seed(seed)

In [3]:
LABESSIZE = 14
plt.rcParams['axes.labelsize'] = LABESSIZE
plt.rcParams['xtick.labelsize'] = LABESSIZE
plt.rcParams['ytick.labelsize'] = LABESSIZE

In [4]:
nn = 6
weights = nx.to_numpy_array(nx.complete_graph(nn))

In [5]:
parameters = {
    "G": 0.55,                         # global coupling strength
    "dt": 0.01,                         # for mpr model [ms]
    "dt_bold": 0.001,                   # for Balloon model [s]
    "J": 14.5,                          # model parameter
    "eta": -4.6*np.ones(nn),                        # model parameter
    "tau": 1.0,                         # model parameter
    "delta": 0.7,                       # model parameter
    "decimate": 500,                    # sampling from mpr time series
    "noise_amp": 0.037,                 # amplitude of noise
    "iapp": 0.0,                        # constant applyed current
    "t_initial": 0.0,                   # initial time * 10[ms]
    "t_transition": 20_000.0,           # transition time * 10 [ms]
    "t_end": 250_000.0,                 # end time * 10 [ms]
    "weights": weights,                 # weighted connection matrix
    "seed": seed,                       # seed for random number generator
    "noise_seed": True,                 # fix seed for noise
    "record_step": 10,                  # sampling every n step from mpr time series
    "output": "output",                 # output directory
    "RECORD_AVG": 0                     # true to store large time series in file
}

In [6]:
control_dict = {
    "eta": {"indices": [[0], [2, 3]], "value": [-4.3, -4.5]}, # set different values for eta
    "G": {"value": 0.5},
    }

In [7]:
obj = MPR_sde(parameters)
# print(obj())
sol = obj.run(par=control_dict)
print(obj.eta)

[-4.3 -4.6 -4.5 -4.5 -4.6 -4.6]


In [8]:
t = sol["t"]
x = sol["x"]

In [9]:
print(f"t.shape = {t.shape}")
print(f"x.shape = {x.shape}")

t.shape = (460,)
x.shape = (6, 460)


In [10]:
if x.ndim == 2:
    pass
    fig, ax = plt.subplots(1, figsize=(10, 3))
    ax.set_xlabel("Time [s]")
    ax.set_ylabel("BOLD")
    plt.plot(t/1000, x.T, alpha=0.8, lw=2)
    plt.margins(0,0.1)
    plt.tight_layout()
    plt.savefig("output/mpr_sde_ts.png", dpi=300)
    plt.close()
else:
    exit(0)

Feature extraction ------------------------------------------------

In [11]:
from vbi.feature_extraction.features_settings import *
from vbi.feature_extraction.calc_features import *

/home/ziaee/anaconda3/envs/vbi_paper/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
fs = 1/(parameters["dt_bold"]) / 1000
cfg = get_features_by_domain(domain="statistical")
# report_cfg(cfg)
data = dataframe_feature_extractor([x], fs, fea_dict=cfg, n_workers=1)
print(data.values)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.77it/s]

   variance_0  variance_1  variance_2  variance_3  variance_4  variance_5  \
0    0.001227    0.002823    0.002001    0.002924    0.002269    0.001861   

     calc_rss     std_0     std_1     std_2  ...  median_abs_deviation_0  \
0  209.154999  0.035029  0.053131  0.044738  ...                 0.02309   

   median_abs_deviation_1  median_abs_deviation_2  median_abs_deviation_3  \
0                0.027323                0.028376                0.031003   

   median_abs_deviation_4  median_abs_deviation_5  hmm_dur_0  hmm_dur_1  \
0                0.027883                 0.02873        0.0       0.25   

   hmm_dur_2  hmm_dur_3  
0        1.0       1.25  

[1 rows x 59 columns]
